In [17]:
import numpy as np
from scipy.stats import norm

In [18]:
class GBM:

    def __init__(self,S0,r,sigma=None):
        self.S0=S0
        self.r=r
        self.sigma=sigma


In [19]:
class Call:

    def __init__(self,K,T):
        self.K=K
        self.T=T

    #You will probably want to insert another function here, to support the control variate calculation in (b)

In [20]:
class CallOnAverage:

    def __init__(self,K,T0,T1):
        self.K = K
        self.T0 = T0
        self.T1 = T1

In [56]:
class MC:

    def __init__(self,M,seed,useControl):
        self.M = M  #number of paths
        self.rng = np.random.default_rng(seed=seed) # Seeding the random number generator with a specified number helps make the calculations reproducible
        self.useControl = useControl

    def randomLogreturn(self,dynamics,deltat):
        return (dynamics.r-dynamics.sigma**2/2)*deltat + dynamics.sigma*np.sqrt(deltat)*self.rng.normal(size=self.M)

    def price_CallOnAverage_GBM(self,contract,dynamics):
    
        #You fill this in.  If self.useControl is False, then this should do part (a), otherwise it should do part (b)
        S05 = dynamics.S0 * np.exp(self.randomLogreturn(dynamics,contract.T0))
        S10 = S05 * np.exp(self.randomLogreturn(dynamics,contract.T1-contract.T0))
        zeropayoff = np.zeros(self.M)
        payoff = np.exp(-dynamics.r*contract.T1) * np.maximum((S05+S10)/2-contract.K,zeropayoff)
        price = np.mean(payoff)
        standard_error = np.std(payoff)/np.sqrt(self.M)
        
        def BScallPrice(sigma,S,rGrow,r,K,T):
            F=S*np.exp(rGrow*T)
            sd = sigma*np.sqrt(T)
            d1 = np.log(F/K)/sd+sd/2
            d2 = d1-sd
            return np.exp(-r*T)*(F*norm.cdf(d1)-K*norm.cdf(d2))
    
            
        if self.useControl:
            Strike = 12
            T = 1

            control_price = BScallPrice(dynamics.sigma, dynamics.S0, dynamics.r, dynamics.r, Strike, T)
            control_payoff = np.exp(-dynamics.r*contract.T1) * np.maximum(S10-Strike,zeropayoff)
        
            cov = np.cov(payoff, control_payoff)
            beta = cov[0,1] / cov[1,1]
            Y_con = payoff + beta*(control_price - control_payoff)
            call_price = np.mean(Y_con)
            standard_error = np.std(Y_con) / np.sqrt(self.M)

            correlation = np.corrcoef(payoff, control_payoff)[0, 1]
            print("Correlation between the contract and the control variate: ", correlation)
    
        

        return(price, standard_error)
        

In [57]:
p3dynamics = GBM(sigma=0.70,S0=10,r=0.02)

In [58]:
p3contract = CallOnAverage(K=12,T0=0.5,T1=1.0)

In [59]:
p3aMC = MC(M=100000,seed=0,useControl=False) 
(price3a, standard_error3a) = p3aMC.price_CallOnAverage_GBM(p3contract,p3dynamics)
print(price3a, standard_error3a)

1.5735749676914146 0.013133445215084201


In [60]:
p3bMC = MC(M=100000,seed=0,useControl=True)
(price3b, standard_error3b) = p3bMC.price_CallOnAverage_GBM(p3contract,p3dynamics)
print(price3b, standard_error3b)

Correlation between the contract and the control variate:  0.9395339840925212
1.5735749676914146 0.0044976229113177614
